#### Queries - Exploración de Tablas DuckLake
##### Configuración
> Asegúrate de tener un archivo `.env` en esta carpeta con las credenciales necesarias.


In [1]:
# Instalar dependencias si es necesario
# !pip install duckdb python-dotenv

In [2]:
import os
import duckdb
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv(dotenv_path='.env', override=True)

# Configuración desde variables de entorno
POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT', '5432')
POSTGRES_DB = os.getenv('POSTGRES_DB')
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

S3_ENDPOINT = os.getenv('S3_ENDPOINT', 'rustfs:9000')
RUSTFS_USER = os.getenv('RUSTFS_USER', 'admin')
RUSTFS_PASSWORD = os.getenv('RUSTFS_PASSWORD')
RUSTFS_SSL = os.getenv('RUSTFS_SSL', 'false')
RUSTFS_BUCKET = os.getenv('RUSTFS_BUCKET', 'mitma')

print("✅ Variables de entorno cargadas")


✅ Variables de entorno cargadas


In [3]:
# Crear conexión a DuckDB
con = duckdb.connect()

# Instalar y cargar extensiones necesarias
critical_extensions = ['ducklake', 'postgres', 'httpfs']
for ext in critical_extensions:
    try:
        con.execute(f"INSTALL {ext};")
        con.execute(f"LOAD {ext};")
        print(f"✅ Extensión {ext} cargada")
    except Exception as e:
        print(f"⚠️ Advertencia al cargar {ext}: {e}")
        try:
            con.execute(f"LOAD {ext};")
            print(f"✅ Extensión {ext} cargada (ya estaba instalada)")
        except Exception as e2:
            print(f"❌ Error al cargar {ext}: {e2}")

# Cargar extensión spatial (opcional)
try:
    con.execute("INSTALL spatial;")
    con.execute("LOAD spatial;")
    print("✅ Extensión spatial cargada")
except Exception as e:
    print(f"⚠️ Spatial extension no disponible: {e}")


✅ Extensión ducklake cargada
✅ Extensión postgres cargada
✅ Extensión httpfs cargada
✅ Extensión spatial cargada


In [4]:
# Configurar S3 para RustFS
con.execute(f"SET s3_endpoint='{S3_ENDPOINT}';")
con.execute(f"SET s3_access_key_id='{RUSTFS_USER}';")
con.execute(f"SET s3_secret_access_key='{RUSTFS_PASSWORD}';")
con.execute(f"SET s3_use_ssl={RUSTFS_SSL};")
con.execute("SET s3_url_style='path';")

print("✅ Configuración de S3/RustFS completada")


✅ Configuración de S3/RustFS completada


In [5]:
# Attach DuckLake con PostgreSQL Catalog
postgres_connection_string = f"""
    dbname={POSTGRES_DB} host={POSTGRES_HOST} user={POSTGRES_USER} password={POSTGRES_PASSWORD} port={POSTGRES_PORT} 
    sslmode=require connect_timeout=30 keepalives=1 keepalives_idle=30 keepalives_interval=10 keepalives_count=5 tcp_user_timeout=30000
"""

# Verificar si ducklake ya está attachado
try:
    databases = con.execute("SELECT database_name FROM duckdb_databases();").fetchdf()
    if 'ducklake' not in databases['database_name'].values:
        attach_query = f"""
            ATTACH 'ducklake:postgres:{postgres_connection_string}' AS ducklake (DATA_PATH 's3://{RUSTFS_BUCKET}/');
        """
        con.execute(attach_query)
        print("✅ DuckLake attachado correctamente")
    else:
        print("✅ DuckLake ya está attachado")
except Exception as e:
    print(f"⚠️ Advertencia al attachar DuckLake: {e}")

# Usar la base de datos ducklake
con.execute("USE ducklake;")
print("✅ Usando base de datos ducklake")

✅ DuckLake attachado correctamente
✅ Usando base de datos ducklake


In [6]:
# Wrapper SQL para simplificar consultas
class SQL:
    """Wrapper para ejecutar queries SQL de forma simple"""
    def __init__(self, connection):
        self.con = connection
    
    def __call__(self, query):
        """Ejecuta una query y retorna el resultado como DataFrame (se muestra automáticamente en notebooks)"""
        return self.fetch_df(query)
    
    def fetch_df(self, query):
        """Ejecuta una query y retorna un DataFrame"""
        return self.con.execute(query).fetch_df()

# Crear instancia global SQL
SQL = SQL(con)
print("✅ Wrapper SQL creado. Usa SQL(query) para ejecutar consultas - el DataFrame se mostrará automáticamente")


✅ Wrapper SQL creado. Usa SQL(query) para ejecutar consultas - el DataFrame se mostrará automáticamente


In [7]:
print("📊 Listando TODAS las tablas en la base de datos:")
SQL("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'main'
    ORDER BY table_name;
""")

📊 Listando TODAS las tablas en la base de datos:


,table_name
0,_temp_od_enriched
1,bronze_ine_empresas_municipio
2,bronze_ine_municipios
3,bronze_ine_poblacion_municipio
4,bronze_ine_renta_municipio
5,bronze_mitma_ine_relations
6,bronze_mitma_municipios
7,bronze_mitma_od_municipios
8,bronze_mitma_overnight_stay_municipios
9,bronze_mitma_people_day_municipios


In [8]:
# Filtrar tablas que comienzan con 'silver_' usando SQL wrapper
SQL("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'main'
        AND table_name LIKE 'silver_%'
    ORDER BY table_name;
""")

,table_name
0,silver_ine_all
1,silver_mitma_distances
2,silver_mitma_ine_mapping
3,silver_od
4,silver_od_all
5,silver_od_quality
6,silver_overnight_stay
7,silver_people_day
8,silver_zones
9,silver_zones_all


## SELECT de cada tabla Silver para ver sus datos


In [9]:
# silver_zones
SQL("SELECT * FROM silver_zones LIMIT 10")

,id,nombre,geometry_obj,centroid
0,01002,Amurrio,"[3, 4, 0, 0, 0, 0, 0, 0, 40, 48, 62, 192, 19, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,01010,Ayala/Aiara,"[3, 4, 0, 0, 0, 0, 0, 0, 122, 1, 69, 192, 93, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,01017_AM,Campezo/Kanpezu agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 156, 226, 27, 192, 23...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,01028_AM,Labastida/Bastida agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 110, 253, 43, 192, 63...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,01036,Laudio/Llodio,"[3, 4, 0, 0, 0, 0, 0, 0, 156, 143, 62, 192, 12...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
5,01043,Oyón-Oion,"[3, 4, 0, 0, 0, 0, 0, 0, 251, 178, 27, 192, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
6,01051,Agurain/Salvatierra,"[3, 4, 0, 0, 0, 0, 0, 0, 227, 202, 24, 192, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
7,01901,Iruña Oka/Iruña de Oca,"[3, 4, 0, 0, 0, 0, 0, 0, 249, 107, 52, 192, 34...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
8,02003,Albacete,"[3, 4, 0, 0, 0, 0, 0, 0, 40, 243, 241, 191, 23...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
9,02008_AM,Alcaraz agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 82, 99, 24, 192, 151,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [10]:
# silver_od
SQL("SELECT * FROM silver_od LIMIT 10")


,fecha,origen_zone_id,destino_zone_id,viajes,viajes_km,distancia,residencia
0,2023-03-02 05:00:00,14005_AM,14053,2.597,32.062,10-50,14
1,2023-03-02 05:00:00,15002,15078,3.128,20.642,2-10,15
2,2023-03-02 05:00:00,17215_AM,17067_AM,5.087,99.206,10-50,17
3,2023-03-02 05:00:00,18023,18021,8.927,753.444,>50,18
4,2023-03-02 05:00:00,20902,20073,4.854,5.279,0.5-2,20
5,2023-03-02 05:00:00,21042,21042,9.639,64.135,2-10,21
6,2023-03-02 05:00:00,24212_AM,37069_AM,2.550,394.724,>50,24
7,2023-03-02 05:00:00,25058,25058,27.328,33.474,0.5-2,25
8,2023-03-02 05:00:00,26002_AM,01043,5.188,142.480,10-50,26
9,2023-03-02 05:00:00,26021_AM,26018,3.356,12.370,2-10,26


In [11]:
# silver_od_all
SQL("SELECT * FROM silver_od_all LIMIT 10")


,fecha,origen_zone_id,destino_zone_id,viajes,viajes_km,distancia
0,2022-03-02,26036,26036,2.660,1.782,0.5-2
1,2022-03-02,27026,27057,4.583,36.584,2-10
2,2022-03-02,28006,28009,3.328,39.985,10-50
3,2022-03-02,28006,28134,31.023,36.494,0.5-2
4,2022-03-02,28007,28074,3.879,15.433,2-10
5,2022-03-02,28007,28079,15.457,105.119,2-10
6,2022-03-02,28014,28014,18.190,57.107,2-10
7,2022-03-02,28047,28068,2.144,3.158,0.5-2
8,2022-03-02,28054,28054,3.861,13.913,2-10
9,2022-03-02,28058,28058,15.121,56.771,2-10


In [12]:
# silver_overnight_stay
SQL("SELECT * FROM silver_overnight_stay LIMIT 10")


,fecha,zona_pernoctacion,zona_residencia,personas
0,2023-03-02,01001,01001,2783.719
1,2023-03-02,01054_AM,01001,2.847
2,2023-03-02,01059,01001,107.508
3,2023-03-02,01901,01001,2.901
4,2023-03-02,08019,01001,9.023
5,2023-03-02,18087,01001,2.280
6,2023-03-02,20045,01001,6.540
7,2023-03-02,28074,01001,2.423
8,2023-03-02,29067,01001,2.901
9,2023-03-02,31060,01001,2.423


In [13]:
# silver_people_day
SQL("SELECT * FROM silver_people_day LIMIT 10")


,fecha,zona_pernoctacion,edad,sexo,numero_viajes,personas
0,2023-03-02,01001,0-25,hombre,0,145.200
1,2023-03-02,01001,0-25,hombre,1,13.200
2,2023-03-02,01001,0-25,hombre,2,67.664
3,2023-03-02,01001,0-25,hombre,2+,225.878
4,2023-03-02,01001,0-25,mujer,0,78.877
5,2023-03-02,01001,0-25,mujer,2,124.257
6,2023-03-02,01001,0-25,mujer,2+,196.199
7,2023-03-02,01001,25-45,hombre,0,81.228
8,2023-03-02,01001,25-45,hombre,2,88.330
9,2023-03-02,01001,25-45,hombre,2+,163.457


In [14]:
# silver_mitma_distances
SQL("SELECT * FROM silver_mitma_distances LIMIT 10")


,origin,destination,distance_km
0,01001,01002,55.649471
1,01001,01004_AM,72.234920
2,01002,01004_AM,18.071794
3,01001,01009_AM,10.761558
4,01002,01009_AM,62.166049
5,01004_AM,01009_AM,77.634455
6,01001,01010,68.728430
7,01002,01010,13.085857
8,01004_AM,01010,8.272757
9,01009_AM,01010,75.073495


In [15]:
# silver_mitma_ine_mapping
SQL("SELECT * FROM silver_mitma_ine_mapping LIMIT 10")


,nombre,codigo_ine,municipio_mitma
0,alegria dulantzi,01001,01001
1,amurrio,01002,01002
2,aramaio,01003,01058_AM
3,artziniega,01004,01004_AM
4,arminon,01006,01047_AM
5,arratzua ubarrundia,01008,01058_AM
6,asparrena,01009,01009_AM
7,aiara,01010,01010
8,manueta,01011,01028_AM
9,barrundia,01013,01009_AM


In [16]:
# silver_ine_empresas_municipio
SQL("SELECT * FROM silver_ine_empresas_municipio LIMIT 10")


CatalogException: Catalog Error: Table with name silver_ine_empresas_municipio does not exist!
Did you mean "bronze_ine_empresas_municipio"?

LINE 1: SELECT * FROM silver_ine_empresas_municipio LIMIT 10
                      ^

In [ ]:
# silver_ine_poblacion_municipio
SQL("SELECT * FROM silver_ine_poblacion_municipio LIMIT 10")


,zone_id,poblacion_total,poblacion_hombres,poblacion_mujeres
0,02034_AM,4320.0,2213.0,2107.0
1,10020_AM,42232.0,20966.0,21266.0
2,04045_AM,5565.0,2880.0,2685.0
3,10004_AM,2769.0,1396.0,1373.0
4,06002,5307.0,2627.0,2680.0
...,...,...,...,...
95,18035_AM,2828.0,1560.0,1268.0
96,18006,7388.0,4014.0,3374.0
97,18119_AM,5008.0,2550.0,2458.0
98,23002,21496.0,10599.0,10897.0


In [ ]:
# silver_ine_renta_municipio
SQL("SELECT * FROM silver_ine_renta_municipio LIMIT 10")


,zone_id,tipo,renta_media
0,02008_AM,renta neta media por persona,12494.0
1,02012_AM,renta neta media por persona,11873.0
2,02024_AM,renta neta media por persona,11708.0
3,02025,renta neta media por persona,11296.0
4,02030,renta neta media por persona,11528.0
...,...,...,...
95,10021,renta neta media por persona,11587.0
96,10205_AM,renta neta media por persona,11090.0
97,10113_AM,renta neta media por persona,11912.0
98,10049_AM,renta neta media por persona,10793.0


In [ ]:
# silver_ine_all
SQL("SELECT * FROM silver_ine_all LIMIT 10")


,id,nombre,empresas,renta_media,poblacion_total,poblacion_hombres,poblacion_mujeres,year
0,01001,Alegría-Dulantzi,150.0,16429.0,2955.0,1522.0,1433.0,2023
1,01002,Amurrio,575.0,17486.0,10364.0,5133.0,5231.0,2023
2,01004_AM,Artziniega agregacion de municipios,163.0,15323.0,3052.0,1545.0,1507.0,2023
3,01009_AM,Asparrena agregacion de municipios,257.0,18615.0,4588.0,2418.0,2170.0,2023
4,01010,Ayala/Aiara,163.0,17163.0,2921.0,1489.0,1432.0,2023
...,...,...,...,...,...,...,...,...
95,03093,Novelda,1601.0,12116.0,26606.0,13204.0,13402.0,2023
96,03094,"Nucia, la",1460.0,12585.0,19121.0,9675.0,9446.0,2023
97,03095,Ondara,675.0,11763.0,7717.0,3855.0,3862.0,2023
98,03096,Onil,567.0,12943.0,8114.0,4091.0,4023.0,2023


In [ ]:
# silver_zones_all
SQL("SELECT * FROM silver_zones_all LIMIT 10")


,id,nombre,geometry_obj,centroid
0,01001,Alegría-Dulantzi,"[3, 4, 0, 0, 0, 0, 0, 0, 175, 184, 32, 192, 24...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,01058_AM,Legutio agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 174, 76, 39, 192, 187...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,01004_AM,Artziniega agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 211, 225, 68, 192, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,02033_AM,Fuente-Álamo agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 4, 30, 182, 191, 201,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,02030,Elche de la Sierra,"[3, 4, 0, 0, 0, 0, 0, 0, 13, 31, 6, 192, 142, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
5,02072_AM,Socovos agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 140, 75, 247, 191, 23...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
6,02069,"Roda, La","[3, 4, 0, 0, 0, 0, 0, 0, 147, 173, 12, 192, 18...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
7,03031,Benidorm,"[3, 4, 0, 0, 0, 0, 0, 0, 93, 227, 3, 190, 63, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
8,03034_AM,Benijófar agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 13, 28, 64, 191, 167,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
9,03055,Catral,"[3, 4, 0, 0, 0, 0, 0, 0, 36, 46, 76, 191, 207,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [17]:
SQL("SELECT * FROM silver_od LIMIT 10")

,fecha,origen_zone_id,destino_zone_id,viajes,viajes_km,distancia,residencia
0,2023-03-02 05:00:00,14005_AM,14053,2.597,32.062,10-50,14
1,2023-03-02 05:00:00,15002,15078,3.128,20.642,2-10,15
2,2023-03-02 05:00:00,17215_AM,17067_AM,5.087,99.206,10-50,17
3,2023-03-02 05:00:00,18023,18021,8.927,753.444,>50,18
4,2023-03-02 05:00:00,20902,20073,4.854,5.279,0.5-2,20
5,2023-03-02 05:00:00,21042,21042,9.639,64.135,2-10,21
6,2023-03-02 05:00:00,24212_AM,37069_AM,2.550,394.724,>50,24
7,2023-03-02 05:00:00,25058,25058,27.328,33.474,0.5-2,25
8,2023-03-02 05:00:00,26002_AM,01043,5.188,142.480,10-50,26
9,2023-03-02 05:00:00,26021_AM,26018,3.356,12.370,2-10,26


In [18]:
SQL("SELECT * FROM bronze_mitma_od_municipios LIMIT 10")

,fecha,periodo,origen,destino,distancia,actividad_origen,actividad_destino,estudio_origen_posible,estudio_destino_posible,residencia,renta,edad,sexo,viajes,viajes_km,loaded_at,source_file
0,20220301,00,01001,01009_AM,2-10,casa,frecuente,no,no,01,10-15,NA,NA,10.279,67.832,2025-12-15 23:49:06.599958+01:00,https://movilidad-opendata.mitma.es/estudios_b...
1,20220301,00,01001,01009_AM,2-10,frecuente,casa,no,no,01,>15,NA,NA,4.591,42.419,2025-12-15 23:49:06.599958+01:00,https://movilidad-opendata.mitma.es/estudios_b...
2,20220301,00,01001,01058_AM,10-50,frecuente,no_frecuente,no,no,01,>15,NA,NA,4.773,72.081,2025-12-15 23:49:06.599958+01:00,https://movilidad-opendata.mitma.es/estudios_b...
3,20220301,00,01001,01058_AM,2-10,frecuente,casa,no,no,01,>15,NA,NA,1.739,15.513,2025-12-15 23:49:06.599958+01:00,https://movilidad-opendata.mitma.es/estudios_b...
4,20220301,00,01001,01059,10-50,frecuente,casa,no,no,01,>15,0-25,mujer,12.318,155.587,2025-12-15 23:49:06.599958+01:00,https://movilidad-opendata.mitma.es/estudios_b...
5,20220301,00,01001,01059,10-50,frecuente,casa,no,no,01,>15,25-45,hombre,3.831,41.863,2025-12-15 23:49:06.599958+01:00,https://movilidad-opendata.mitma.es/estudios_b...
6,20220301,00,01001,01059,10-50,frecuente,casa,no,no,01,>15,45-65,mujer,2.961,31.749,2025-12-15 23:49:06.599958+01:00,https://movilidad-opendata.mitma.es/estudios_b...
7,20220301,00,01001,01059,10-50,frecuente,casa,no,no,01,>15,NA,NA,1.0,10.132,2025-12-15 23:49:06.599958+01:00,https://movilidad-opendata.mitma.es/estudios_b...
8,20220301,00,01002,01002,0.5-2,casa,frecuente,no,no,01,10-15,0-25,hombre,13.052,10.225,2025-12-15 23:49:06.599958+01:00,https://movilidad-opendata.mitma.es/estudios_b...
9,20220301,00,01002,01002,0.5-2,casa,frecuente,no,no,01,10-15,45-65,NA,4.021,3.286,2025-12-15 23:49:06.599958+01:00,https://movilidad-opendata.mitma.es/estudios_b...


In [ ]:
# Cerrar la conexión cuando termines
# con.close()
# print("✅ Conexión cerrada")
